# 引入库

In [ ]:
import keras
import os
import json
import shutil
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from keras import backend as K
from mylibs.ProcessBar import ShowProcess
from mylibs import funs
from mylibs.my_contrib import *
keras.__version__

# 猫狗分类实验
## 实验目的
vgg16模型调参


## 实验数据说明
  * 网络数据下载地址：https://www.kaggle.com/c/dogs-vs-cats/data
  * 本地数据存储路径：~/e/dataset_tiptical/cats_and_dogs
  * 实验数据根目录：~/data/cats_and_dogs
      - ./ori      ------------猫狗分类原始数据目录
      - ./lab_base ------------实验方案目录
      - ./lab_base/train ------训练目录
      - ./lab_base/valid ------校验目录
      - ./lab_base/test  ------测试目录
      - ./lab_base/test.jpg ---测试图片
      

## 参考资料
visualization of filters keras 基于Keras的卷积神经网络（CNN）可视化

http://www.cnblogs.com/bnuvincent/p/9612686.html

python深度学习{eep learning with python中文版.pdf}源码

https://github.com/fchollet/deep-learning-with-python-notebooks

数据下载：

https://www.kaggle.com/c/dogs-vs-cats/data

本地数据

~/e/dataset_tiptical/cats_and_dogs


# 实验2
预训练模型

## 实验参数

In [ ]:
##实验参数
print('\n==============================================')
print('设置实验参数')
lab_name='猫狗分类实验2'                                #实验名称
data_path='%s/data/cats_and_dogs'%(os.getenv('HOME')) #猫狗分类数据根目录
ori_path='%s/ori'%(data_path)                         #猫狗分类原始文件目录
lab_path='%s/lab_vgg16_fine_tuning'%(data_path)       #实验方案目录
split_num="10000,2000,2000"                           #实验数据分割方案,<1：比例分割，>1：数量分割
batch_size=32                                         #批量大小
data_enhance=False                                    #ImageDataGenerator数据启用数据增强
epochs=10                                             #训练轮次
img_width=150                                         #训练图像宽度
img_height=150                                        #训练图像高度 
test_img_path='%s/test.jpg'%(data_path)               #测试图片路径
images_per_row=16       #图像显示每行显示的单元个数
#feature_map_top_num=12  #FeatureMap前面N层{include_top=False}
img_margin=3            #图像单元空隙
layers_name=['conv2d_1','conv2d_2','conv2d_3','conv2d_4'] #卷积层名称
#layers_name=['conv2d_1'] #卷积层名称
last_conv_layer_name='conv2d_4' #最后一层卷积层
gen_pat_steps=40                           #构造迭代次数
cp_file='%s/checkpoint.h5'%(lab_path)      #ModelCheckpoint 文件路径
his_file='%s/history.json'%(lab_path)      #训练日志文件路径
class_mode='binary'                        #分类方法,'binary':二分类，'categorical':多分类
loss='binary_crossentropy'  #损失函数,'binary_crossentropy':二分类，'categorical_crossentropy':多分类

test_cat_path='%s/test_cat.jpg'%(data_path) #猫的测试图像
test_dog_path='%s/test_dog.jpg'%(data_path) #狗的测试图像

## 加载数据

In [ ]:
##加载数据
print('\n==============================================')
print('加载数据......')
#数据生成器-测试数据集(测试集数据不能打乱)
test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(
        '%s/test2'%(data_path),
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle=False, #测试集数据不能打乱
        class_mode=class_mode)

# VGG16

In [ ]:
from keras.applications import VGG16

In [ ]:
## 创建网络
model_vgg16=VGG16(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
model = models.Sequential()
model.add(model_vgg16)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
#打印模型
model.summary()
#模型编译
model.compile(loss=loss,
          optimizer=optimizers.RMSprop(lr=1e-4),
          metrics=['acc'])

In [ ]:
## 网络预测
preds=model_vgg16.predict_generator(
    test_gen, 
    steps=None, #预测轮数
    max_queue_size=32, 
    workers=1, 
    use_multiprocessing=False,     
    verbose=1)

# VGG19


# ResNet50


# Inception V3


# Xception


# MobileNet

## 创建网络

In [ ]:
##构建网络
print('\n==============================================')
print('构建网络')
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(img_height, img_width, 3),name='conv2d_1'))
model.add(layers.MaxPooling2D((2, 2),name='max_pooling2d_1'))
#model.add(layers.core.Dropout(0.5))
model.add(layers.Conv2D(64, (3, 3), activation='relu',name='conv2d_2'))
model.add(layers.MaxPooling2D((2, 2),name='max_pooling2d_2'))
#model.add(layers.core.Dropout(0.5))
model.add(layers.Conv2D(128, (3, 3), activation='relu',name='conv2d_3'))
model.add(layers.MaxPooling2D((2, 2),name='max_pooling2d_3'))
#model.add(layers.core.Dropout(0.5))
model.add(layers.Conv2D(128, (3, 3), activation='relu',name='conv2d_4'))
model.add(layers.MaxPooling2D((2, 2),name='max_pooling2d_4'))
#model.add(layers.core.Dropout(0.5))
model.add(layers.Flatten(name='flatten_1'))
model.add(layers.Dense(512, activation='relu',name='dense_1'))
model.add(layers.Dense(1, activation='sigmoid',name='dense_2'))    
#打印模型
model.summary()
#模型编译
model.compile(loss=loss,
          optimizer=optimizers.RMSprop(lr=1e-4),
          metrics=['acc'])

## 网络训练

In [ ]:
class train_callback(keras.callbacks.Callback):
    def __init__(self,log_file,history={},verbose=0):
        super(train_callback,self).__init__() #调用父类构造函数
        self.log_file=log_file #训练日志文件路径
        self.history=history   #训练日志
        self.verbose=verbose   #是否显示保存信息
        
    #on_epoch_end: 在每个epoch结束时调用
    def on_epoch_end(self,epoch,logs=None):
        #最佳日志
        if len(self.history)==0:
            for k,v in logs.items():
                self.history[k]=[v]
        else:
            for k,v in logs.items():
                self.history[k].append(v)
        #保存日志
        json.dump(self.history,open(self.log_file,'w'))
        if self.verbose==1:
            print('更新训练日志(%d):%s'%(len(self.history),self.log_file))

In [ ]:
##网络训练
print('\n==============================================')
print('网络训练 ......')
#加载断点
if os.path.exists(cp_file):
    model.load_weights(cp_file)
    print('加载模型文件:',cp_file)
#训练日志
history2={}
if os.path.exists(his_file):
    history2=json.load(open(his_file,'r'))
    print('加载训练日志:',his_file)

In [ ]:
#回调函数保存训练日志    
his_cb=train_callback(his_file,history=history2)

#断点训练:monitor监控参数可以通过self.score = self.model.evaluate(self.x_test, self.y_test, verbose=0)的score查询
checkpoint_cb = ModelCheckpoint(cp_file, monitor='val_acc', verbose=1, save_best_only=True, mode='auto',period=2)
#EarlyStopping
earlyStopping_cb=keras.callbacks.EarlyStopping(monitor='acc', patience=3, verbose=0, mode='max')
#TensorBoard
#tensorBoard_cb=TensorBoard(log_dir=self.log_dir)
#回调函数序列
callbacks_list = [checkpoint_cb,earlyStopping_cb,his_cb]

history = model.fit_generator(
  train_gen,
  steps_per_epoch=np.ceil(train_gen.samples/batch_size),
  epochs=epochs,
  validation_data=valid_gen,
  validation_steps=50,
  callbacks=callbacks_list)

## 保存模型

In [ ]:
##保存模型
print('\n==============================================')
print('保存模型 ......') 
save_model(model,lab_path,train_gen.class_indices)

# 测试简报

In [ ]:
score_valid = model.evaluate_generator(valid_gen, steps=100, max_q_size=10, workers=1, pickle_safe=False,verbose=1)

In [ ]:
preds=model.predict_generator(
    test_gen, 
    steps=None, #预测轮数
    max_queue_size=32, 
    workers=1, 
    use_multiprocessing=False,     
    verbose=1)
print(valid_gen.samples)

In [ ]:
preds_acc_val=preds_acc(preds,test_gen)

In [ ]:
print('\n\n')
print('================测试简报(%s[%s])=================='%(lab_name,lab_path))
#网络模型
model.summary()
#网络输入
print('input_shape:%s'%(model.input.shape))
#网络训练
print('train=>epochs :%d,samples:%d,loss:%f,acc:%f'
          %(epochs,len(train_gen.filenames),history.history['loss'][-1],history.history['acc'][-1]))
#网络评估
print('valid=>samples:%d,val_loss:%f,val_acc:%f'%(len(valid_gen.filenames),score_valid[0],score_valid[1]))
#网络测试
print('test =>samples:%d,acc:%f'%(len(test_gen.filenames),preds_acc_val))

## 网络测试

In [ ]:
images=[]
funs.GatherFilesEx('%s/test'%(data_path),images,exts='.jpg,.jpeg')
predict_images(model,images,train_gen.class_indices,img_width=img_width,img_height=img_height)

## 训练曲线

In [ ]:
##训练曲线
print('\n==============================================')
print('训练曲线')
visualizer_scalar(history.history)
visualizer_scalar(history2)

## 可视化FeatureMap

In [ ]:
##可视化FeatureMap
print('\n==============================================')
print('可视化FeatureMap-猫')
visualizer_feature_map(model,test_cat_path,target_size=(img_height,img_width)
                       ,images_per_row=images_per_row,img_margin=img_margin)

In [ ]:
print('可视化FeatureMap-狗')
visualizer_feature_map(model,test_dog_path,target_size=(img_height,img_width)
                       ,images_per_row=images_per_row,img_margin=img_margin)

## 可视化类激活热力图

In [ ]:
##可视化类激活热力图
print('可视化类激活热力图-猫')
visualizer_heatmap(model,test_cat_path,last_conv_layer_name,target_size=(img_height,img_width))

In [ ]:
##可视化类激活热力图
print('可视化类激活热力图-狗')
visualizer_heatmap(model,test_dog_path,last_conv_layer_name,target_size=(img_height,img_width))

## 可视化网络过滤器

In [ ]:
##可视化网络过滤器
print('\n==============================================')
print('可视化网络过滤器')
visualizer_filter_input(model,layers_name,gen_pat_steps=gen_pat_steps,images_per_row=images_per_row
                        ,img_width=img_width,img_height=img_height,img_margin=img_margin)